# Prototypowy model

Ten notebook implementuje model ExtraTreesClassifier do przewidywania jakości powietrza w uproszczonej formie:
- Klasa `1`: Good + Moderate
- Klasa `0`: Poor + Hazardous

## Import bibliotek i wczytanie danych

In [16]:
# Import bibliotek
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Wczytanie danych
file_path = 'C:\\Users\\shind\\Documents\\GitHub\\s26102-Air-Quality-And-Pollution\\data\\pollution_dataset.csv'
data = pd.read_csv(file_path)

# Przekształcenie zmiennej Air Quality na klasy binarne
data['Air Quality Binary'] = data['Air Quality'].map(
    lambda x: 1 if x in ['Good', 'Moderate'] else 0
)

# Usuwanie niepotrzebnych kolumn
X = data.drop(columns=['Air Quality', 'Air Quality Binary', 'PM10'])  # PM10 jako redundantna
y = data['Air Quality Binary']

# Podział na zbiory treningowe i walidacyjne
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Rozkład klas w zbiorach
print("Rozkład klas w zbiorze treningowym:\n", y_train.value_counts())
print("Rozkład klas w zbiorze walidacyjnym:\n", y_val.value_counts())


Rozkład klas w zbiorze treningowym:
 Air Quality Binary
1    2435
0    1065
Name: count, dtype: int64
Rozkład klas w zbiorze walidacyjnym:
 Air Quality Binary
1    1065
0     435
Name: count, dtype: int64


## Inicjalizacja modelu

In [12]:
model = ExtraTreesClassifier(
    bootstrap=True,
    criterion='entropy',
    max_features=0.6,
    min_samples_leaf=20,
    min_samples_split=20,
    n_estimators=100,
    random_state=42
)

## Trenowanie modelu

In [13]:
model.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, criterion='entropy', max_features=0.6,
                     min_samples_leaf=20, min_samples_split=20,
                     random_state=42)

## Ewaluacja modelu

In [14]:
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

print(f"Dokładność na zbiorze walidacyjnym: {accuracy:.2f}")
print("Raport klasyfikacji:")
print(classification_report(y_val, y_pred, target_names=['Poor+Hazardous', 'Good+Moderate'], zero_division=0))

Dokładność na zbiorze walidacyjnym: 0.71
Raport klasyfikacji:
                precision    recall  f1-score   support

Poor+Hazardous       0.00      0.00      0.00       435
 Good+Moderate       0.71      1.00      0.83      1065

      accuracy                           0.71      1500
     macro avg       0.35      0.50      0.42      1500
  weighted avg       0.50      0.71      0.59      1500


### Wyniki ewaluacji

#### **Dokładność (Accuracy):**
- Model osiągnął dokładność na poziomie **71%** na zbiorze walidacyjnym.
- Wysoka dokładność wynika z dominacji klasy `Good+Moderate` (klasa `1`) w zbiorze danych.

---

#### **Raport klasyfikacji (classification_report):**

| Klasa           | Precyzja (Precision) | Czułość (Recall) | F1-score | Liczba próbek (Support) |
|------------------|----------------------|------------------|----------|-------------------------|
| `Poor+Hazardous` | 0.00                | 0.00             | 0.00     | 435                     |
| `Good+Moderate`  | 0.71                | 1.00             | 0.83     | 1065                    |

- **Macro avg:**
  - Precyzja: **0.35**
  - Czułość: **0.50**
  - F1-score: **0.42**
- **Weighted avg:**
  - Precyzja: **0.50**
  - Czułość: **0.71**
  - F1-score: **0.59**

---

### Interpretacja wyników

1. **Klasa `Good+Moderate` (1):**
   - Model doskonale przewiduje tę klasę (czułość 100%), co wynika z jej dominacji w zbiorze danych.
   - Precyzja na poziomie 71% oznacza, że model popełnia błędy w przewidywaniu tej klasy, myląc niektóre próbki z klasą `Poor+Hazardous`.

2. **Klasa `Poor+Hazardous` (0):**
   - Model w ogóle nie przewiduje tej klasy (precyzja, czułość, F1-score wynoszą 0).
   - To wskazuje na problem niezrównoważonych danych — klasa `Poor+Hazardous` jest marginalizowana przez model.

3. **Macro avg:**
   - Niska wartość (0.42) pokazuje, że model działa bardzo nierównomiernie między klasami.

4. **Weighted avg:**
   - Średnia ważona (0.59) jest podbita przez dominującą klasę `Good+Moderate`, co nie odzwierciedla pełnej skuteczności modelu.

---

### Wnioski i rekomendacje

1. **Problem z niezrównoważonymi danymi:**
   - Klasa `Poor+Hazardous` jest znacznie mniej liczna (435 próbek vs 1065 próbek), co prowadzi do ignorowania jej przez model.

2. **Balansowanie klas:**
   - Rozważ zastosowanie technik balansowania, takich jak **undersampling** klasy `Good+Moderate` lub **oversampling** klasy `Poor+Hazardous` (np. SMOTE).



In [15]:
# Import SMOTE
from imblearn.over_sampling import SMOTE

# Zastosowanie SMOTE do balansowania danych
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Sprawdzenie rozkładu klas po balansowaniu
print("Rozkład klas w zbiorze treningowym po SMOTE:\n", pd.Series(y_train_balanced).value_counts())

# Trenowanie modelu na zbalansowanych danych
model.fit(X_train_balanced, y_train_balanced)

# Ewaluacja modelu
y_pred_balanced = model.predict(X_val)
accuracy_balanced = accuracy_score(y_val, y_pred_balanced)

print(f"Dokładność na zbiorze walidacyjnym po SMOTE: {accuracy_balanced:.2f}")
print("Raport klasyfikacji po SMOTE:")
print(classification_report(y_val, y_pred_balanced, target_names=['Poor+Hazardous', 'Good+Moderate'], zero_division=0))

Rozkład klas w zbiorze treningowym po SMOTE:
 Air Quality Binary
1    2435
0    2435
Name: count, dtype: int64
Dokładność na zbiorze walidacyjnym po SMOTE: 0.51
Raport klasyfikacji po SMOTE:
                precision    recall  f1-score   support

Poor+Hazardous       0.26      0.37      0.30       435
 Good+Moderate       0.69      0.57      0.63      1065

      accuracy                           0.51      1500
     macro avg       0.47      0.47      0.47      1500
  weighted avg       0.56      0.51      0.53      1500


### Wyniki ewaluacji po zastosowaniu SMOTE

#### **Dokładność (Accuracy):**
- Model osiągnął dokładność na poziomie **51%** na zbiorze walidacyjnym.
- Jest to spadek w porównaniu do modelu bez SMOTE, co wskazuje na wyzwania związane z balansem między poprawą wyników dla klasy `Poor+Hazardous` a zachowaniem skuteczności dla `Good+Moderate`.

---

#### **Raport klasyfikacji (classification_report):**

| Klasa           | Precyzja (Precision) | Czułość (Recall) | F1-score | Liczba próbek (Support) |
|------------------|----------------------|------------------|----------|-------------------------|
| `Poor+Hazardous` | 0.26                | 0.37             | 0.30     | 435                     |
| `Good+Moderate`  | 0.69                | 0.57             | 0.63     | 1065                    |

- **Macro avg:**
  - Precyzja: **0.47**
  - Czułość: **0.47**
  - F1-score: **0.47**
- **Weighted avg:**
  - Precyzja: **0.56**
  - Czułość: **0.51**
  - F1-score: **0.53**

---

### Interpretacja wyników

1. **Klasa `Good+Moderate` (1):**
   - Precyzja wynosi **69%**, a czułość **57%**. Model nadal skutecznie przewiduje tę klasę, choć jej skuteczność spadła w porównaniu do wyników bez SMOTE.

2. **Klasa `Poor+Hazardous` (0):**
   - Precyzja wynosi **26%**, a czułość **37%**, co stanowi poprawę w porównaniu do modelu bez SMOTE (gdzie wszystkie metryki wynosiły 0).
   - Model wciąż ma trudności z dokładnym klasyfikowaniem tej klasy.

3. **Macro avg i Weighted avg:**
   - Wyniki są bardziej zrównoważone niż w modelu bez SMOTE, co wskazuje na lepsze rozpoznawanie obu klas.

---

### Wnioski i rekomendacje

1. **SMOTE poprawił wyniki dla klasy mniejszościowej (`Poor+Hazardous`):**
   - Czułość wzrosła z 0 do **37%**.
   - Precyzja również wzrosła, choć wciąż jest niska (**26%**).

2. **Negatywny wpływ na ogólną dokładność:**
   - Zbalansowanie danych spowodowało spadek ogólnej dokładności modelu z 71% do **51%**, co wskazuje na trudności w zachowaniu równowagi.

